<h1 align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;数据科学引论 - Python之道 </h1>

<h1 align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第5课 数据收集 - Python网络爬虫实践 I </h1>

# 爬虫概述
在阅读这个样例之前，建议先了解爬虫是什么，简单理解url、爬虫技术、网页html等基本概念。

在完成本笔记本的操作之前，大家可以通过命令 ```pip install scrapy``` 或 ```pip3 install scrapy``` 安装所需依赖包。

# 定义爬虫的任务

## 涉及的语法
语法涉及类（面向对象）、列表list、字典dict、循环、函数、字符串操作、文件读写

## 概述
这个爬虫的任务是爬取http://quotes.toscrape.com/page/1/ 的前2页，提取每条名言的文字内容，作者和标签，最后以JSON格式保存到文件中


## 如何修改

在自己做定制时，只需要修改`__init__`和`parse`两个方法，通俗讲__init__方法决定了爬取哪些网站，parse则指明了在每一个网页上爬取哪些内容
- init_urls: 设置待爬取网站的列表和保存文件路径，其中变量self.urls是待爬取网站的列表，self.file是一个文件对象
- parse：方法内是针对每个url成功访问之后进行的页面解析
   关于如何解析具体网页，也就是选择器的使用，与网页格式十分相关，这个样例无法适用于其他网站。由于选择器的使用有很大的选择性，所以可以参考文档http://scrapy-chs.readthedocs.io/zh_CN/latest/topics/selectors.html


In [ ]:
import scrapy
import time
import json
import os

class MySpider(scrapy.Spider):
    
    name = "spider"
    allowed_domains = ['quotes.toscrape.com']
    
    
    def __init__(self):
        
        self.file = open('demo1_quotes.json', 'w');
        
        #设置待爬取网站列表
        self.urls = []
        for i in range(1,3):
            self.urls.append('http://quotes.toscrape.com/page/' + str(i) + '/')
            
#       初始化效果 效果等同
#         self.urls = [
#             'http://quotes.toscrape.com/page/1/',
#             'http://quotes.toscrape.com/page/2/',
#         ]

        
    def start_requests(self):
        for url in self.urls:
            time.sleep(5)
            yield scrapy.http.Request(url=url, callback=self.parse, dont_filter=True)
    

    #parse方法会在每个request收到response之后调用
    def parse(self, response):
        
        #提取名言列表
        quotes = response.css("div.quote");
        for quote in quotes:
            #提取每条名言中的作者名
            author = quote.css("small.author::text").extract_first();
            #提取名言的文字内容
            text = quote.css(".text::text").extract_first();
            #提取名言标签
            tags = quote.css(".tags a::attr(href)").extract();
            #构建字典对象
            item = {"author":author, "text": text, "tags":tags };
            #将字典转换成json字符串
            line = json.dumps(dict(item))
            #将每个条目写入文件
            self.file.write(line + "\n")
        #及时将内容写入文件，否则可能会出现少许延迟
        self.file.flush()
        os.fsync(self.file)
        #输出当前解析完成的网页网址，可以当做爬取进度来看待,与程序逻辑无关
        print("over: " + response.url)
        
    def close(spider, reason):
        self.file.close();
        

# 执行爬虫任务
启动后，将执行Myspider。
这部分的代码块，如果确实非常了解scrapy的运行机制，那么可以做定制，否则不建议自行修改。

In [ ]:
from scrapy.crawler import CrawlerProcess
from multiprocessing import Process
from scrapy.utils.project import get_project_settings


def run_spider():
    process = CrawlerProcess(get_project_settings())
    process.crawl(MySpider)
    process.start() # 这句代码就是开始了整个爬虫过程 
    
run_spider()

打印一下看看效果：

In [ ]:
data_list = []  # 存储解析后的JSON对象的列表

with open("demo1_quotes.json", "r") as json_file:
    for line in json_file:
        try:
            json_data = json.loads(line)
            data_list.append(json_data)
        except json.JSONDecodeError as e:
            print(f"JSON解析错误：{e}")

# 现在data_list中包含了所有JSON对象的列表
for item in data_list:
    print(item)